In [16]:
import yfinance as yf
import pandas as pd

he = yf.download("HE", start="2023-01-01", end="2023-12-31")[
    ["Open", "High", "Low", "Close", "Volume"]
    ]
"""
he = yf.download("HE", start="2023-01-01", interval="15m")[
    ["Open", "High", "Low", "Close", "Volume"]
]
"""
he.columns = he.columns.droplevel(1)
he.head()

[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Volume
Date,,,,,
2023-01-03,40.105215,40.411580,39.607370,40.153084,293900
2023-01-04,40.306268,41.043461,40.076495,40.947723,485700
2023-01-05,40.631781,40.832836,39.655239,40.057346,524400
2023-01-06,40.536044,41.167924,40.497747,40.823261,339600
2023-01-09,40.784964,40.976444,40.545616,40.641357,415800


In [17]:
from backtesting import Strategy, Backtest
from backtesting.test import SMA
import pandas as pd

def std_3(arr, n):
    return pd.Series(arr).rolling(n).std() * 3

class MeanReversion(Strategy):
    roll = 50
    def init(self):
        self.he = self.data.Close
        self.he_mean = self.I(SMA, self.he, self.roll)
        self.he_std = self.I(std_3, self.he, self.roll)
        self.he_upper = self.he_mean + self.he_std
        self.he_lower = self.he_mean - self.he_std
        self.he_close = self.I(SMA, self.he, 1)
    
    def next(self):
        if self.he_close < self.he_lower:
            self.buy(
                tp = self.he_mean,
            )
        if self.he_close > self.he_upper:
            self.sell(
                tp = self.he_mean,
            )

In [18]:
bt = Backtest(he, MeanReversion, cash=10000, commission=.002)
stats = bt.run()
bt.plot()
stats

Start                     2023-01-03 00:00:00
End                       2023-12-29 00:00:00
Duration                    360 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   4414.04456
Equity Peak [$]                       10000.0
Return [%]                          -55.85955
Buy & Hold Return [%]               -60.61746
Return (Ann.) [%]                   -56.14739
Volatility (Ann.) [%]                41.05659
CAGR [%]                            -43.58616
Sharpe Ratio                         -1.36756
Sortino Ratio                        -0.93363
Calmar Ratio                          -0.8027
Max. Drawdown [%]                   -69.94785
Avg. Drawdown [%]                   -69.94785
Max. Drawdown Duration      141 days 00:00:00
Avg. Drawdown Duration      141 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                   